# phase 1 分完重點後，切換到對應的 thread

step 1 只有一步

In [ ]:
import os
import time
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
# print(OpenAI.api_key)

In [ ]:
import pandas as pd
import numpy as np

csv_file = './data/key_point_embeddings0514.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()

In [ ]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

In [ ]:
def get_similar_QAK(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    key_point = np.array(response.data[0].embedding, ndmin=2)
    distances, indices = index.search(key_point, 3)
    QAK_csv = './data/QAK0514.csv'
    QAK_df = pd.read_csv(QAK_csv)
    QAK_array = QAK_df.to_numpy()
    QAKD = [(QAK_array[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    top3 = '{"Three candidate problem-resolution approaches": [\n'
    for i in range(0,3):
        top3 = top3 + '{"problem": "' + QAKD[i][0][0].replace("\"", "\'") + '",\n'
        top3 = top3 + '"summarized keypoints": "[' + QAKD[i][0][2][16:-3].replace("\"", "\'") + ']",\n'
        top3 = top3 + '"solution": { \n "form": "direct answer", \n "content": "' + QAKD[i][0][1].replace("\"", "\'") + '"}},\n'
    top3 = top3[0:-2] + ']}'
    return top3

def get_similar_flow(input):
    pass

模擬 phase1

In [ ]:
question = ""

In [ ]:
instruction = ""
instructions = [
    "# Objective: ",
    "As an intelligent customer service chatbot of the “Flora” focus app , my goal is to understand and help you solve problems encountered when using flora app effectively. " ,
    "# Main features of the Flora app: ",
    "Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what’s more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track.* * Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most. ",
    "# Support Process: ",
    "Issue Identification: To ensure that I understand your problems, I will summarize the constructive, non-emotional parts of our input text in your tone into one key point. but if one key point is not enough to describe problem, key point can be more than one. Avoid using conjunctions. Then, I will and must only output these key points in json format. ",
    "#Example: ",
    "input: How do I change my password? ",
    'output: {"keypoints": ["my problem is I need to change password."]}',
    "input: I am just doing my a-levels at the moment and I love using your app for revision. However, when I went onto the app earlier it logged me out and all my data had been lost. I was just wondering if you could help me out.",
    'output: {"keypoints": ["My data lost after being logged out of the app."]}'
    "input: My flora friend page won't load. If I delete the app and re download it and then login. Will all my data and garden time be lost?",
    'output: {"keypoints": ["my Flora friend page will not load.","I am concern about losing data and garden time after deleting and reinstalling the app."]}'
]
for i in instructions:
    instruction = instruction + i + "\n"

模擬 phase1 拿到 key point

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": instruction},
        {"role": "user", "content": question}
    ]
)
time.sleep(5)
key_point_json = completion.choices[0].message.content.replace("\n", " ")
print(key_point_json)

In [ ]:
import json

key_point = []
try:
    for i in range(0,len(json.loads(key_point_json)["keypoints"])):
        key_point.append(str(json.loads(key_point_json)["keypoints"][i]))

except json.JSONDecodeError as e:
    key_point = key_point_json
    print("phase1 didn't pass json format")

print(key_point)

模擬 phase2

In [ ]:
threadNum = 0

instruction = ""
instructions = [
    "# Objective ",
    "Act as an intelligent customer service chatbot for the 'Flora' focus app. Below are: A. Background of the app, B. the past conversation log with the user, obtained from the “Feedback Portal” system, C. Three candidate problem-resolution approaches. Each approach specifies a problem, a problem summarization (key point), and a solution in the form of either direct answer or resolution flow.",
    "Your job is: 1. Decide which candidate problem-resolution approach in C can be applied to solve the user's main problem. Note that the conversation log in B may contain irrelevant information and you should only use it as a clue for solving user's main problem. First,If user's main problem not related to Flora, you are prohibited to answer and must reply only a special mark:'*'. Second,If three problem-resolution approaches especially summarized keypoints within not related to user's main problem, you are prohibited to answer and must reply only a special mark:'*'. third, If there are multiple related approaches, choose the one that is the most suitable.2. Based on the chosen problem-resolution approach, chat with the user to solve her main problem without any format. If the approach contains an answer directly, follow the answer to interact with the user. You must not deviate from the chat based on the answer. If the approach contains a flow, execute the flow by interacting with the user and calling functions accordingly. You must follow the flow and ask users to stay on track if they deviate from the flow.3. If the user asks another question different from user's main problem either during or after the problem resolution process in Step 2, prompt the user to go back to the “Feedback Portal” system to officially file the problem.",
    "# A. Main features of the Flora app",
    "Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what's more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track.* * Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most.",
    "# B.Past Conversation Log"
]
for i in instructions:
    instruction = instruction + i + "\n"

partB = '{"Past Conversation Log": [{"role": "user", "message": "' + question + '"}, {"role": "assistant", "message": "This is my summarized key points:'+ str(key_point) + ',and i will focus on main problem and help you solve it."}]}'
instruction = instruction + partB + "\n"

instruction += "# C. Candidate Problem-resolution Approaches \n"
main_problem = key_point[threadNum]
top3 = get_similar_QAK(main_problem)
instruction += top3

print(instruction)

In [ ]:
assistant = client.beta.assistants.create(
    name="customer service chatbot",
    instructions=instruction,
    model="gpt-3.5-turbo-1106"
)
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= "User's Main Problem:"+main_problem
  )

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
  )
while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )

In [ ]:
output = client.beta.threads.messages.list(thread_id=thread.id,order='desc').data[0].content[0].text.value
print(output)

In [ ]:
response = client.beta.assistants.delete(assistant.id)
time.sleep(3)
print(response)